In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

class Hist_2D:
    def __init__(self, bins, binRange=None):
        _, self.binsX, self.binsY = np.histogram2d([-10,10], [-10,10], bins=bins, range=binRange)
        self.val = np.zeros((len(self.binsX)+1,len(self.binsY)+1))
        self.err = np.zeros((len(self.binsX)+1,len(self.binsY)+1))
        
    def isNumber(self, val):
        return isinstance(val, (float, np.floating, int, np.integer))
    
    def getBinRangeX(self):
        return arr([self.binsX[0],self.binsX[-1]])
    # def getRangeY(self):
    #     return self.binsY[1:-2]
    # def get_range(self):
    #     return self.binsX[1:-2], self.binsY[1:-2]
    
    def getBinNX(self):
        return len(self.binsX)-1
    def getBinNY(self):
        return len(self.binsY)-1
    def getBinN(self):
        return self.binNX()*self.binNY()\
            
    def getBinWidthX(self, binX=1):
        return self.binsX[binX+1]-self.binsX[binX]
    def getBinWidthY(self, binY=1):
        return self.binsY[binY+1]-self.binsY[binY]
            
    def setBin(self, binX, binY, value, err=None): 
        self.val[binX,binY] = value
        if err==None:
            self.poisson_err_(binX, binY)
        else:
            self.err[binX, binY] = err
            
    def getBinsX(self):
        return self.binsX
            
    def getBin(self, x, y):
        return self.getBinCol(x), self.getBinRow(y)
    def getBinCol(self, x):
        try:
            return arr(np.where(np.logical_and(x>=self.binsX[:-1], x<self.binsX[1:]))).flatten()[0]+1
        except:
            return (0 if x<self.binsX[0] else len(self.binsX))
    def getBinRow(self, y):
        try:
            return arr(np.where(np.logical_and(y>=self.binsY[:-1], y<self.binsY[1:]))).flatten()[0]+1
        except:
            return (0 if y<self.binsY[0] else len(self.binsY))
    
    def fill(self, x, y, w=1):
        if self.isNumber(x) and self.isNumber(y):
            binX = self.getBinCol(x)
            binY = self.getBinRow(y)
            self.val[binX, binY] += w
            self.poisson_err_(binX, binY)
            return
        if len(x)!=len(y):
            print("[ERROR] Hist_2D.fill(): x and y need to have the same length. The lengths are:", len(x), len(y))
            return
        for i in range(len(x)):
            binX = self.getBinCol(x[i])
            binY = self.getBinRow(y[i])
            self.val[binX, binY] += w
            self.poisson_err_(binX, binY)
    
    def getBinContent(self, col, row):
        return self.val[col, row]
        
    def hist(self):
        return self.val[1:-1, 1:-1]
    def histErr(self):
        return np.sqrt(self.err[1:-1, 1:-1])
    
    def calcErr_poisson(self, binX, binY):
        self.err[binX,binY] = np.sqrt(abs(self.val[binX,binY]))
        
    def draw(self, ax, extent=None, cmap="viridis", cbar_label=None, overflow=False, **kwargs):
        if not overflow:
            X = self.hist()
            im = ax.imshow(self.hist().T, extent=[self.binsX[0],self.binsX[-1],self.binsY[0],self.binsY[-1]], origin='lower', **kwargs)
        else: 
            X = self.val
            widthX = self.getBinWidthX(0)
            widthY = self.getBinWidthY(0)
            im = ax.imshow(X.T, extent=[self.binsX[0]-widthX, self.binsX[-1]+widthX, self.binsY[0]-widthY, self.binsY[-1]+widthY], origin='lower', **kwargs)
            ax.plot([self.binsX[0],self.binsX[-1],self.binsX[-1],self.binsX[0],self.binsX[0]], [self.binsY[0],self.binsY[0],self.binsY[-1],self.binsY[-1],self.binsY[0]], color='black', linestyle=":")
            
        cbar_ax = inset_axes(ax, width="5%", height="100%", loc = 'lower left',
                    bbox_to_anchor = (1.02, 0., 1, 1), bbox_transform = ax.transAxes,
                    borderpad = 0)
        cbar = plt.colorbar(im, cax=cbar_ax)
        cbar.set_label(cbar_label, loc="top")
        cbar_ax.tick_params(axis="y", direction="in")
        return im

    def printOverflow(self):
        print("Overflow:")
        
        print(uf(self.val[0,-1], self.err[0,-1]), "|", uarr(self.val[1:-1,0], self.err[1:-1,0]).sum(), "|", uf(self.val[-1,-1], self.err[-1,-1]))
        print(uarr(self.val[0,1:-1], self.err[0,1:-1]).sum(), "|", uarr(self.val[1:-1,1:-1], self.err[1:-1,1:-1]).sum(), "|", uarr(self.val[-1,1:-1], self.err[-1,1:-1]).sum())

class Plot_2D(Hist_2D):
    # len(bins) == nBins+1
    # len(data) == nBins+2
    def __init__(self, bins, binRange=None, mode="Mean"):
        super().__init__(bins, binRange)
        self.mode = mode
        self.data = np.empty([len(self.binsX)+1,len(self.binsY)+1], dtype=object)
        for binX in range(len(self.binsX)+1):
            for binY in range(len(self.binsY)+1):
                self.data[binX, binY] = arr([])
                
    def getBinData(self, col, row):
        return self.data[col, row]
    
    
    def fill_entries(self, xs, ys, zs, MPV_binN=None, MPV_binRange=None, MPV_removeTail=0.95):
        # dataset of tuples of (x,y,z). x,y define the bin, z is averaged for each bin
        if not ((len(xs)==len(ys)) and (len(ys)==len(zs))):
            print("[ERROR] Plot_2D(): fill_entries needs x,y,z arrays of the same lengths. The lengths are:", len(xs), len(ys), len(zs))
        xs = [self.getBinCol(x) for x in xs]
        ys = [self.getBinRow(y) for y in ys]
        for i in range(len(xs)):
            self.data[xs[i],ys[i]] = np.append(self.data[xs[i],ys[i]], [zs[i]])
        self.fill_hist_(MPV_binN, MPV_binRange, MPV_removeTail)
        
    def fill_hist_(self, MPV_binN=None, MPV_binRange=None, removeTail=0.95):
        for binX in range(len(self.binsX)+1):
            for binY in range(len(self.binsY)+1):
                if len(self.data[binX, binY]>0):
                    if self.mode == "Mean":
                        temp = self.getBinMean(binX, binY, removeTail)
                        val = temp.n
                        err = temp.s
                    elif self.mode == "MPV":
                        temp = self.getBinMPV(binX, binY, MPV_binN, MPV_binRange, removeTail)
                        val = temp.n
                        err = temp.s
                    self.setBin(binX, binY, val, err**2)
                else: 
                    self.setBin(binX, binY, 0.,0.)
    
    def getBinMean(self, col, row, removeTail=0.95):
        entries = self.data[col,row]
        if removeTail is not None:
            entries.sort()
            entries = entries[:int(len(entries)*removeTail)]
        v = np.mean(entries)
        e = np.std(entries)
        return uf(v,e)
        
    def getBinMPV(self, col, row, binN=None, binRange=None, removeTail=0.95):
        entries = self.data[col,row]
        if removeTail is not None:
            entries.sort()
            entries = entries[:int(len(entries)*removeTail)]
        if binN is None and binRange is None:
            hist, bins = np.histogram(entries, bins=max(int(np.sqrt(len(entries))),100))
        elif binN is None and binRange is not None:
            hist, bins = np.histogram(entries, bins=max(int(np.sqrt(len(entries))),100), range=binRange)
        else:
            hist, bins = np.histogram(entries, bins=binN, range=binRange)
        v = bins[np.where(hist == hist.max())[0][0]] + (bins[1]-bins[0]) / 2
        e = (bins[1]-bins[0]) / 2
        return uf(v,e)
    
def uzarr(shape):
    return unp.uarray(np.zeros(shape), np.zeros(shape))

In [ ]:
nEvents = 10000000
runIDs = arr([184,190])

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, nEvents*4)
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],nEvents)
    
M = runs[runIDs[0]]["M"]

In [ ]:
binRange = [0,0]
binN = 40
bin = 3

fig, ax = Hist.create_fig(1,1, [6,4])

bin_entries = hist.getBinData(bin+1, slice_row+1)
bin_entries.sort()
top90 = int(len(bin_entries)*0.95)
bin_entries = bin_entries[:top90]

binRange[1] = max(bin_entries)

bin_hist, bin_bins = np.histogram(bin_entries, bins=binN, range=binRange)

ax.stairs(bin_hist, bin_bins, fill=True, color="black", alpha=0.5)
ax.stairs(bin_hist, bin_bins, fill=False, color="black")

mu = bin_entries.mean()
sig = bin_entries.std()
ax.axvline(mu, color="red", label="Mean (trunc. @ 95%)")
ax.axvline(mu+sig, ls=":", color="red", label="Std (trunc. @ 95%)")
ax.axvline(mu-sig, ls=":", color="red")

ax.set_xlim(binRange)


# x_range = [hist.binX[bin], hist.]
Text = "{:.2f} < x [pix] < {:.2f}".format(hist.binsX[bin], hist.binsX[bin+1]) + "\n" + "{:.2f} < y [pix] < {:.2f}".format(hist.binsY[bin], hist.binsY[bin+1])
patch = mpl.patches.Patch(color='white', label=Text)
handles, labels = ax.get_legend_handles_labels()
handles.append(patch)


Hist.finalize(runs[runID], fig, ax, title="Risetime", xlabel="risetime [ns]", ylabel="Entries", param_narrow=True, param_fontsize=6)
ax.legend(handles=handles)

In [ ]:
FileName = "Risetime_MeanStd"
# runID = 192
# zRange = [5,175]
runIDs = arr([184,190])
zRange = [5,40]
binN = arr([14,10])
cols = [M["TrkCol"], M["TrkRow"], M["Risetime"]]
cm = mpl.colormaps["viridis"]
%matplotlib inline

slice_row = 3

def draw_colorbar(ax, label=None, width="5%", height="100%", **kwargs):
    cbar_ax = cbar_ax = inset_axes(ax, width=width, height=height, loc = 'lower left',
                   bbox_to_anchor = (1.02, 0., 1, 1), bbox_transform = ax.transAxes,
                   borderpad = 0)
    cbar = plt.colorbar(im, cax=cbar_ax, **kwargs)
    cbar.set_label(label, loc="top")
    return cbar_ax, cbar

for runID in runIDs:
    mask = runs[runID]["data"][:,:,M["Amplitude"]] > 0
    mask = np.logical_and(mask, runs[runID]["data"][:,:,M["Risetime"]] > 0)
    entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])
    binRange = arr([[-0.5,1.5], [-0.5,1.5]])

    hist = Plot_2D(binN, binRange, mode="Mean")
    hist.fill_entries(entries[0], entries[1], entries[2])

    fig, ax = Hist.create_fig(1,1, figsize=[10,4])
    
    ax.hlines(hist.binsY[slice_row], binRange[0][0], binRange[0][1], color="red", linestyle=":", zorder=10000)
    ax.hlines(hist.binsY[slice_row+1], binRange[0][0], binRange[0][1], color="red", linestyle=":", zorder=10000)
    ax.vlines(binRange[0,0], hist.binsY[slice_row], hist.binsY[slice_row+1], color="red", linestyle=":", zorder=10000)
    ax.vlines(binRange[0,1], hist.binsY[slice_row], hist.binsY[slice_row+1], color="red", linestyle=":", zorder=10000)
    im = hist.draw(ax, extent=binRange.flatten(), cbar_label="Risetime [ns]", overflow=True, vmin=zRange[0], vmax=zRange[1])
    
    ax.set_aspect(25/35)
    ax.grid(False)
    ax.set_xticks([0,1])
    ax.set_yticks([0,1])
    
    axs = fig.get_axes()
    
    Hist.finalize(runs[runID], fig, ax, title="Inpix Risetime\n(mean+std, trunc. 95%)", xlabel="x [pix]", ylabel="y [pix]", param_narrow=True, param_fontsize=6)
    ax.grid(False)
    fig.savefig("output/Inpixel/"+FileName+"_"+str(runID)+".pdf", bbox_inches='tight')
    
    fig, ax = Hist.create_fig(1,1, figsize=[5,4])

    slice_bins = hist.binsX
    slice_hist = hist.val[1:-1,slice_row+1]
    slice_hist_e = np.sqrt(hist.err[1:-1,slice_row+1])
    slice_binWidth = hist.getBinWidthX()
    slice_xVal = slice_bins[:-1] + slice_binWidth/2
    
    ax.stairs(slice_hist, slice_bins, fill=True, color="black", alpha=0.5)
    ax.stairs(slice_hist, slice_bins, fill=False, color="black", alpha=1)
    ax.errorbar(slice_xVal, slice_hist, yerr=slice_hist_e, color="black", linestyle="", capsize=3)
    ax.set_xlim(hist.getBinRangeX())
       
    Hist.finalize(runs[runID], fig, ax, title="Inpix Risetime\n(mean+-std, trunc. 95%)", xlabel="x [pix]", ylabel="Risetime [ns]", param_narrow=True, param_fontsize=6)
    ax.grid(False)

In [ ]:
FileName = "Risetime_MPV"
# runID = 192
# zRange = [5,175]
runIDs = arr([184,190])
zRange = [5,40]
binN = arr([14,10])
cols = [M["TrkCol"], M["TrkRow"], M["Risetime"]]
cm = mpl.colormaps["viridis"]
%matplotlib inline

slice_row = 1


def draw_colorbar(ax, label=None, width="5%", height="100%", **kwargs):
    cbar_ax = cbar_ax = inset_axes(ax, width=width, height=height, loc = 'lower left',
                   bbox_to_anchor = (1.02, 0., 1, 1), bbox_transform = ax.transAxes,
                   borderpad = 0)
    cbar = plt.colorbar(im, cax=cbar_ax, **kwargs)
    cbar.set_label(label, loc="top")
    return cbar_ax, cbar

for runID in runIDs:
    mask = runs[runID]["data"][:,:,M["Amplitude"]] > 0
    mask = np.logical_and(mask, runs[runID]["data"][:,:,M["Risetime"]] > 0)
    entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])
    binRange = arr([[-0.5,1.5], [-0.5,1.5]])

    hist = Plot_2D(binN, binRange, mode="MPV")
    hist.fill_entries(entries[0], entries[1], entries[2], MPV_binRange=[0,50], MPV_binN=100)

    fig, ax = Hist.create_fig(1,1, figsize=[10,4])
    
    ax.hlines(hist.binsY[slice_row], binRange[0][0], binRange[0][1], color="red", linestyle=":", zorder=10000)
    ax.hlines(hist.binsY[slice_row+1], binRange[0][0], binRange[0][1], color="red", linestyle=":", zorder=10000)
    ax.vlines(binRange[0,0], hist.binsY[slice_row], hist.binsY[slice_row+1], color="red", linestyle=":", zorder=10000)
    ax.vlines(binRange[0,1], hist.binsY[slice_row], hist.binsY[slice_row+1], color="red", linestyle=":", zorder=10000)
    im = hist.draw(ax, extent=binRange.flatten(), cbar_label="Risetime [ns]", overflow=True, vmin=zRange[0], vmax=zRange[1])
    
    ax.set_aspect(25/35)
    ax.grid(False)
    ax.set_xticks([0,1])
    ax.set_yticks([0,1])
    
    axs = fig.get_axes()
    
    Hist.finalize(runs[runID], fig, ax, title="Inpix Risetime\n(MPV)", xlabel="x [pix]", ylabel="y [pix]", param_narrow=True, param_fontsize=6)
    ax.grid(False)
    fig.savefig("output/Inpixel/"+FileName+"_"+str(runID)+".pdf", bbox_inches='tight')
    
    fig, ax = Hist.create_fig(1,1, figsize=[5,4])
    
    slice_bins = hist.binsX
    slice_hist = hist.val[1:-1,slice_row+1]
    slice_hist_e = np.sqrt(hist.err[1:-1,slice_row+1])
    slice_binWidth = hist.getBinWidthX()
    slice_xVal = slice_bins[:-1] + slice_binWidth/2
    
    # ax.stairs(slice_hist, slice_bins, fill=True, color="black", alpha=0.5)
    # ax.stairs(slice_hist, slice_bins, fill=False, color="black", alpha=1)
    ax.errorbar(slice_xVal, slice_hist, xerr=slice_binWidth/2 ,yerr=slice_hist_e, color="black", linestyle="", capsize=3, markersize=2)
    ax.set_xlim(hist.getBinRangeX())
    ax.set_ylim(ax.get_ylim()[0], ax.get_ylim()[1]*1.2)
    
    
    Hist.finalize(runs[runID], fig, ax, title="Inpix Risetime\n(MPV)", xlabel="x [pix]", ylabel="Risetime [ns]", param_narrow=True, param_fontsize=6)
    
    Text = "errorbars are binwidth\nin histogram for\nMPV-ing"
    patch = mpl.patches.Patch(color='white', label=Text, alpha=0)
    handles, labels = ax.get_legend_handles_labels()
    handles.append(patch)
    ax.legend(handles=handles, loc="upper left")
    ax.grid(False)

In [ ]:
# hitmap
runID = 190
binN = [14,10]
cols = [M["TrkCol"], M["TrkRow"]]

mask = runs[runID]["data"][:,:,M["TrkResX"]] < 15
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["TrkResY"]] < 15)
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixInCluster"]] == 1)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

binRange = [[min(entries[0]), max(entries[0])],[min(entries[1]),max(entries[1])]]
binRange = [[-0.5,1.5],[-0.5,1.5]]

hitmap = Hist_2D(binN, binRange)
hitmap.fill(entries[0], entries[1])

fig, ax = Hist.create_fig(1,1, figsize=[10.,4.])

hitmap.draw(ax, cmap="viridis", extent=[0,35,0,25], overflow=True)

ax.set_xticks([0,1])
ax.set_yticks([0,1])
ax.set_aspect(25/35)

Hist.finalize(runs[runID], fig, ax, title="Hitmap", xlabel="x [pix]", ylabel="y [pix]", param_narrow=True, param_fontsize=6)


In [ ]:
Name = "Falltime"
runID = 184
shareCoords = False
fig, axs = plt.subplots(1, 1, figsize=(4,3), sharex=shareCoords, sharey=shareCoords)

binN = 20
binRange = [0,600]
# binRange = None
column = M[Name]

# mask = np.any(runs[runIDs[0]]["data"][:,:,M["Charge"]] > 2000, axis=1)
pixWithHighestCharge = np.argmax(runs[runIDs[0]]["data"][:,:,M["Charge"]], axis=1)

for i_pix in range(4):
    # mask = runs[runIDs[0]]["data"][:,i_pix,M["Charge"]] >= 220
    # print(mask)
    # mask = i_pix == pixWithHighestCharge
    entries = runs[runIDs[0]]["data"][:,i_pix, column]
    entries = entries[entries != 0]
    hist, bins = np.histogram(entries, bins=binN, range=binRange)
    
    Hist.draw(axs, hist, bins, color="C"+str(i_pix), fill_alpha=0.1, label="Pix "+str(i_pix+1)+" (N={:.1f}k)".format(np.sum(hist)/1000))
    
axs.set_xlim(binRange)
Hist.finalize(runs[runID], fig, axs, Name, "Entries", Name, measurement="TB", logy=False)

In [ ]:
runID = 184

rangeX = [0, 50]
rangeY = [0, 300]
binNX = 20
binNY = 20
NameX = "Risetime"
NameY = "Falltime"
shareCoords = True
logScale = True

colX = M[NameX]
colY = M[NameY]
entriesX = runs[runID]["data"][:,i_pix,colX]
entriesY = runs[runID]["data"][:,i_pix,colY]

pixWithHighestCharge = np.argmax(runs[runIDs[0]]["data"][:,:,M["Charge"]], axis=1)


# hist, binsx, binsy = np.histogram2d(a, b)

# fig, axs = plt.subplots(2,2, figsize=(6, 4))

fig, axs = Hist.create_fig(2,2, [8,6], sharex=shareCoords, sharey=shareCoords)

for i_pix in range(4):
	# mask = i_pix == pixWithHighestCharge
	mask = len(runs[runID]["data"][:,i_pix,colX])*[True]
	# mask = np.logical_and(runs[runID]["data"][mask,i_pix,colX] != 0, runs[runID]["data"][mask,i_pix,colY] != 0) # remove 0 entries
	# mask = runs[runID]["data"][mask,i_pix,colY] != 0 # remove 0 entries
	entriesX = runs[runID]["data"][mask,i_pix,colX]
	entriesY = runs[runID]["data"][mask,i_pix,colY]
	hist, binsx, binsy = np.histogram2d(entriesX, entriesY, bins=[binNX, binNY], range=[rangeX, rangeY])

	extent = [binsx[0], binsx[-1], binsy[0], binsy[-1]]

	if not logScale:
		axs[i_pix].imshow(hist.T, extent=extent, aspect="auto", origin='lower', cmap='viridis')
	else:
		axs[i_pix].imshow(hist.T, extent=extent, aspect="auto", origin='lower', norm=LogNorm(), cmap='viridis')

# lt.imshow(H, interpolation='nearest', origin='lower',

        # extent=[xedges[0], xedges[-1], yedges[0, yedges[-1]])

Hist.finalize(runs[runID], fig, axs, title=NameY+" vs "+NameX, xlabel=NameX, ylabel=NameY)

In [ ]:
%matplotlib inline
# %matplotlib qt

runID = 184
binN = [14,10]
cols = [M["TrkCol"], M["TrkRow"]]

cm = mpl.colormaps["viridis"]

mask = runs[runID]["data"][:,:,M["TrkResX"]] < 15
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["TrkResY"]] < 15)
mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixInCluster"]] == 1)
entries = arr([runs[runID]["data"][:,:,col].flatten()[mask.flatten()] for col in cols])

binRange = [[min(entries[0]), max(entries[0])],[min(entries[1]),max(entries[1])]]
binRange = [[-0.5,1.5],[-0.5,1.5]]

hist, binsX, binsY = np.histogram2d(entries[0], entries[1], bins=binN, range=binRange)
print(binsX)
print(binsX[:-1]-(binsX[1]-binsX[0])/2)

barEdgesX = binsX[:-1]
barEdgesY = binsY[:-1]
barWidthX = binsX[1]-binsX[0]
barWidthY = binsY[1]-binsY[0]


barEdgesArrayX = arr([barEdgesX for i in range(len(barEdgesY))]).T.flatten()
barEdgesArrayY = arr([barEdgesY for i in range(len(barEdgesX))]).flatten()
barEdgesArrayZ = np.zeros_like(hist).flatten()
barWidthArrayX = np.ones_like(hist).flatten()*barWidthX
barWidthArrayY = np.ones_like(hist).flatten()*barWidthY
barWidthArrayZ = hist.flatten()

posArrayX = barEdgesArrayX + barWidthArrayX/2
posArrayY = barEdgesArrayY + barWidthArrayY/2
posArrayZ = hist.flatten()
step = 4
posArrayX = posArrayX[::step]
posArrayY = posArrayY[::step]
posArrayZ = posArrayZ[::step]


fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection="3d")

ax.bar3d(barEdgesArrayX, barEdgesArrayY, barEdgesArrayZ, barWidthArrayX, barWidthArrayY, barWidthArrayZ, color=cm(hist.flatten()/np.max(hist.flatten())), zsort='average', alpha=.6)
# ax.plot_wireframe(posArrayX.reshape(hist.shape), posArrayY.reshape(hist.shape), hist, color="black", alpha=0.7)
ax.errorbar(posArrayX, posArrayY, posArrayZ, zerr=np.sqrt(posArrayZ.flatten()), fmt=".", color="black", capsize=2, markersize=1, elinewidth=1, zorder=1000000, barsabove=True)


if False:
    # xpos, ypos = np.meshgrid(xedges[:-1]+xedges[1:], yedges[:-1]+yedges[1:])

    print(xpos)

    x_centers = arr([(xedges[:-1] + xedges[1:]) / 2 for i in range(len(yedges)-1)]).T
    y_centers = arr([(yedges[:-1] + yedges[1:]) / 2 for i in range(len(xedges)-1)])
    # coords = np.array([x_centers, y_centers])

    # x_centers = xpos.T
    # y_centers = ypos.T

    xpos = xpos.flatten()/2.
    ypos = ypos.flatten()/2.
    zpos = np.zeros_like (xpos)

    dx = xedges [1] - xedges [0]
    dy = yedges [1] - yedges [0]
    dz = hist.flatten()

    # x_centers -= dx/2
    # y_centers -= dy/2

    # x_centers = arr([xpos + dx/2 for i in range(len(ypos))])
    # y_centers = arr([ypos + dy/2 for i in range(len(xpos))])
    max_height = np.max(dz)   # get range of colorbars so we can normalize
    min_height = np.min(dz)
    # scale each z to [0,1], and get their rgb values
    rgba = [cm((k-min_height)/max_height) for k in dz] 

    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111, projection="3d")

    # ax.errorbar(x_centers.flatten(), y_centers.flatten(), hist.flatten(), zerr=np.sqrt(hist.flatten()), fmt="none", ecolor="black", capsize=2)
    # ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color=rgba, zsort='average', alpha=0.7)
    print(x_centers.shape, y_centers.shape, hist.shape)
    ax.plot_wireframe(x_centers, y_centers, hist, color="black", alpha=0.7)
    ax.errorbar(xpos+dx/2, ypos+dy/2, dz, zerr=np.sqrt(hist.flatten()), fmt="None", ecolor="black", capsize=2)

    # ax.set_aspect(25/35)\
    
if True:
    ax.set_box_aspect([35,25,30])
    plt.title("ER1 Hitmap")
    plt.xlabel("x [pix]", loc="right")
    plt.ylabel("y [pix]", loc="top")
    plt.savefig("Your_title_goes_here")
    plt.show()
    # plt.clf()
    